# Thinking1:什么是监督学习，无监督学习，半监督学习

1. 监督学习
- 带有标签的学习
2. 无监督学习
- 没有标签的学习
3. 半监督学习（Semi-Supervised learning):
- 半监督学习介于监督学习和无监督学习之间
- 通常半监督学习的任务与监督学习一致，即任务中包含有明确的目标（如分类），采用的数据包括有标签的数据，也包括无标签的数据
- 作用：只有少量的数据有 Label,利用没有标签的数据来学习整个数据的潜在分布

# Thinking2:K-means 中的k值如何选取？ 

- 数据的先验知识，或者数据进行简单的分析
- 基于变化的算法:给定一个合理的类簇指标，比如平均半径或者直径，只要我们假设的类簇的数目等于或者高于真实的类簇的数目时，该指标上升缓慢，而一旦试图得到少于真实数目的类簇时，该指标会急剧上升。
- 基于结构的算法：即比较类内距离、类间距离以确定K
- 基于一致性矩阵的算法
- 基于层次聚合
- 基于采样的算法
- 使用Canopy Method 算法初始化

# Thinking3:随机森林采用了 bagging 集成学习，bagging 指的是什么

将若干个弱分类器的分类结果进行投票选择，从而组成一个强分类器，这就是随机森林的 bagging 思想,实际上bagging的思想是’机器学习加强版‘。

# Thinking4:表征学习和半监督学习的区别是什么

1. 表征学习：
- 定义：表征学习（representation),也称为特征学习（feature learning），目的是对复杂的原始数据化繁为简，把原始的无效信息剔除，把有效信息更有效地进行提炼，形成特征



2. 半监督学习：
- 定义：通常半监督学习的任务与监督学习一致，即任务中包含有明确的目标（如分类），采取的数据既包括有标签的数据，也包括无标签的数据
- 作用：只有少量的数据有Label,利用没有标签的数据来学习整个数据的潜在分布

表征学习针对的是整个数据集，而半监督学习针对的是对没有标签的数据

In [45]:
# 文本抄袭自动检测分析
import numpy as np 
import pandas as pd 
import os 
from collections import defaultdict

from sklearn import preprocessing
from sklearn.preprocessing import Normalizer  
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

import jieba
import editdistance
import pickle  

In [46]:
# 加载停用词
with open('chinese_stopwords.txt','r',encoding = 'utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]  

In [47]:
# 数据加载
news = pd.read_csv('sqlResult.csv',encoding = 'gb18030')  

In [48]:
# 处理缺失值
news = news.dropna(subset = ['content'])  

In [49]:
# 分词（在写 TF-IDF之前需要进行一下分词）
def split_text(text):
    text = text.replace(' ','')  
    text = text.replace('\n','')  
    text2 = jieba.cut(text.strip())  
    result = ' '.join([w for w in text2 if w not in stopwords])  
    
    return result

In [50]:
if not os.path.exists('corpus.pkl'):
    corpus = list(map(split_text,[str(i) for i in news.content]))  
    with open('corpus.pkl','wb') as file: 
        pickle.dump(corpus,file) 
else:
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)

In [51]:
# 计算 corpus 的 TF-IDF矩阵
countvectorizer = CountVectorizer(encoding = 'gb18030',min_df = 0.015) 
tfidftransformer = TfidfTransformer()
countvector = countvectorizer.fit_transform(corpus) 
tfidf = tfidftransformer.fit_transform(countvector)  

In [52]:
# 自己标记是否是自己的新闻
label = list(map(lambda source: 1 if '新华' in str(source) else 0,news.source))  

In [53]:
# 数据集划分
x_train,x_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3,random_state = 33) # random_state 是随机数种子，这个值可以自己设置

#  定义一个多项式的朴素贝叶斯分类器
clf = MultinomialNB()
clf.fit(x_train,y_train)
prediction = clf.predict(tfidf.toarray())  # 这里的 tfidf.toarray是全量数据

labels = np.array(label)  
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels}) 
# 计算所有可疑文章的 index 
copy_news_index = compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels']  == 0)].index
print('可疑文章数：',len(copy_news_index))
# 计算所有新华社文章的 index
xinhuashe_news_index = compare_news_index[(compare_news_index['labels'] == 1)].index

可疑文章数： 2828


In [54]:
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())  

if not os.path.exists('label.pkl'):
    # 使用 kmeans ,对全量文档做聚类
    kmeans = KMeans(n_clusters = 25)  
    k_labels = kmeans.fit_predict(scaled_array)
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels,file)
    print('k_labels.shape',k_labels.shape)
else:
    with open('label.pkl','rb') as file:
        k_labels = pickle.load(file)


In [55]:
# 创建反向的 id_class
if not os.path.exists('id_class.pkl'):
    id_class = {index:class_ for index,class_ in enumerate(k_labels)}
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class,file)
else:
    with open('id_class.pkl','rb') as file:
        id_class = pickle.load(file)

In [56]:
# 创建反向的 class_id
if not os.path.exists('class_id.pkl'):
    class_id = defaultdict(set)  
    for index,class_ in id_class.items():
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id,file)
else:
    with open('class_id.pkl','rb') as file:
        class_id = pickle.load(file)

In [57]:
# 找相似文本
def find_similar_text(cpindex,top = 10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(),key = lambda x:x[1][0],reverse = True)[:top] 

In [58]:
# 统计可疑抄袭文本里面的编辑距离 
dict_ = dict()
for cpindex in copy_news_index:
    similar_list = find_similar_text(cpindex)
    # 找一篇相似的原文
    similar2 = similar_list[0][0]
    # 参看编辑距离
    editdistance_ = editdistance.eval(corpus[cpindex],corpus[similar2])
    dict_[cpindex] = (editdistance_,similar2)

In [59]:
# 把所有可疑抄袭的文章按照 计算出来的编辑距离从小到大进行排序
dict_new = sorted(dict_.items(),key = lambda d:d[1][0])
most_similarity_index = {x[0]:x[1][1] for x in dict_new[:5]}
print(most_similarity_index)

{7968: 46172, 5709: 86644, 2172: 2246, 3794: 1115, 8262: 8447}


In [ ]:
# 查看编码距离最小的五项，即最可能发生抄袭的五篇
for cpindex,similar2 in most_similarity_index.items():

    print('怀疑抄袭:\n',news.iloc[cpindex].content)
    # 找一篇相似的原文
    print('相似原文\n',news.iloc[similar2].content)
    # 参看编辑距离
    print('编辑距离:', dict_[cpindex][0])
    print('**********************************************')